# BioEn Enemble Refinement with DEER Data
(This Jupyter notebook documents the use of BioEn with an ensemble of conformations. For the use of BioEn spin-label reweighting, please either rotamer_refinement_potra.ipynb orrotamer_refinement.ipynb.)

By applying this ipython notebook, we can perform spin-label ensemble refinement with DEER data using BioEn. The steps of the Jupyter notebook are <br> 

1. Preparation of the input <br> 
2. BioEn <br> 
3. Analysis of the BioEn results<br> 

You can either use the provided test data or define your own protein of interest, use your own rotamer library, and define labeled position of interest.


### Theory and methods applied in this Jupyter notebook are available in:

K. Reichel, Lukas S. Stelzl, J. Köfinger, G. Hummer, “Precision DEER Distances from Spin-Label Reweighting“, J. Phys. Chem. Lett. (2018).

G. Hummer, J. Köfinger, “Bayesian ensemble refinement by replica simulations and reweighting”, J. Chem. Phys. 143, 243150 (2015).

### Experimental DEER traces used in this Jupyter notebook are published in:
R. Dastvan, E. Brouwer, D. Schuetz, O. Mirus, E. Schleiff, T.F. Prisner, ”Relative orientation of POTRA domains from cyanobacterial Omp85 studied by pulsed EPR spectroscopy”, Biophys. J. 110, 2195--2206} (2016).

In [ ]:
import numpy as np
import pickle
import pandas as pd

import MDAnalysis as mda
import MDAnalysis.analysis.align
import MDAnalysis.lib.NeighborSearch as KDNS
import MDAnalysis.analysis.distances

from scipy.special import fresnel

import matplotlib as m
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab

import bioen_rotamer_conformations as brc

%matplotlib inline

## Define paths to the experimental and simulated data and for the output of preparation, BioEn, and analysis of the BioEn runs

In [ ]:
%%capture
! mkdir files/output_preparation
! mkdir files/output_bioen
! mkdir files/output_analysis

# path to experimental data
path_experimental_data = "files/experimental_data/"

# path to simulated data
path_simulated_data = "files/simulated_data/"

# number of ensemble members/conformations
# names of the files are 'conf0-signal-deer.pdb', 'conf1-signal-deer.pdb', ... 'conf9-signal-deer.pdb' 
# in the directory 'files/conformations/
nmodels = 10

# define path to the output of the preparation
path_output_preparation = "files/output_preparation"

# define path to the output of BioEn
path_output_bioen = "files/output_bioen"

# define path to the output of the analysis
path_output_analysis = "files/output_analysis"

# 1. Preparation

## Settings and input parameters for preparation

In [ ]:
# pdb of the label (here: MTSL), trajectory and weights of the rotamer library
rotamer_library_pdb = "files/rotamer_library/rotamer1_R1A_298K_2015.pdb"
rotamer_library_dcd = "files/rotamer_library/rotamer1_R1A_298K_2015.dcd"
rotamer_library_weights = "files/rotamer_library/rotamer1_R1A_298K_weights_2015.dat"

# spin label positions to obtain rotamer states
spin_labels = [[370, 'C'], [292, 'A']]

# spin_label_pairs used for the spin-label reweighting
spin_label_pairs = [[[370, 'C'], [292, 'A']]]

# define clash_distance between atoms of the spin-spin label and the protein
clash_distance = 1.4

## Obtain for each labeled position a trajectory of spin-label rotamers

In [ ]:
def get_rotamers(structure):
    """
    Calculates for each spin-label position possible rotameric states
    according to the protein structure.
       
    Returns
    -------
    rotamer_labels: dict
        For each spin-label position an array with boolean entries 
        for each rotameric state
    """
    xray = mda.Universe(structure)
    rotamers = mda.Universe(rotamer_library_pdb, rotamer_library_dcd)

    rotamer_labels = dict()
    for label in spin_labels:
        
        resid = label[0]
        chainid = label[1]
        dcd_fn = '{}/tmp_traj_{}_{}.dcd'.format(path_output_preparation, resid, chainid)
        d = dict()
        brc.fit_rotamers(rotamers, xray, resid, chainid, dcd_fn)
        rotamer_site = mda.Universe(rotamers.filename, dcd_fn)
        rotamer_clash = brc.find_clashing_rotamers(clash_distance, rotamer_site, xray, resid, chainid)
        d['site'] = rotamer_site
        d['clash'] = rotamer_clash

        rotamer_nitrogen = rotamer_site.select_atoms("name N1")
        rotamer_oxygen = rotamer_site.select_atoms("name O1")
        rotamer_positions = dict()
        for rotamer in rotamer_site.trajectory:
             if not rotamer_clash[rotamer.frame]:
                rotamer_electron = (np.array(rotamer_nitrogen.positions[0][0:3]) + np.asarray(rotamer_oxygen.positions[0][0:3]))/2.0
                rotamer_positions[rotamer.frame] = [rotamer_electron]
        d['positions'] = rotamer_positions
        ln = '{}-{}'.format(resid, chainid)
        rotamer_labels[ln] = d

    return rotamer_labels

## Get for each spin-label distance in the rotamer trajectories a single DEER trace

In [ ]:
def get_preparation_output(rotamer_labels, rotamer_library_weights):
    sim_model = dict()
    distances_model = dict()
    for spin_label_pair in spin_label_pairs:
        label_pair_id = [spin_label_pair[0][0], spin_label_pair[1][0]]
        ts = brc.get_experimental_timesteps(path_experimental_data, label_pair_id)
        spin_label_pair.append(ts)
        ln = "{}-{}".format(spin_label_pair[0][0], spin_label_pair[1][0])
        sim_model[ln], distances_model[ln] = brc.get_deer_from_single_pair_conformations(spin_label_pair, 
                                                                                         rotamer_labels, 
                                                                                         rotamer_library_weights,
                                                                                         path_output_preparation)    
    return sim_model, distances_model

## Get all input for BioEn

In [ ]:
def get_data_ensemble():
    """
    Generates for an ensemble of conformations the DEER traces.
    Saves input for BioEn:
        data_*.pkl --> DEER traces based on spin-label distances
        models_*.dat --> list of the model ids
    """   

    sim_tmp = dict()
    distances_tmp = dict()
    for nmodel in range(0,10):
        structure = "files/conformations/conf{}-signal-deer.pdb".format(nmodel)
        rotamer_labels = get_rotamers(structure)  
        brc.check_rotamer_labels(rotamer_labels, nmodel)      
        sim_tmp[nmodel], distances_tmp[nmodel] = get_preparation_output(rotamer_labels, 
                                                                        rotamer_library_weights)
    # save simulated DEER traces in pkl format (input for bioen)
    data_pkl = "{}/data_input_sim.pkl".format(path_output_preparation)
    with open(data_pkl, 'wb') as fp:
        pickle.dump([sim_tmp], fp) 
        
    # save simulated distances traces in pkl format (input for bioen)
    distances_pkl = "{}/distances.pkl".format(path_output_preparation)
    with open(distances_pkl, 'wb') as fp:
        pickle.dump(distances_tmp, fp) 
    
    np.savetxt("{}/models.dat".format(path_output_preparation), range(0,nmodels))
    return

In [ ]:
nmodels = 10
sim_tmp = get_data_ensemble()

# BioEn

## Settings and input parameters for BioEn

In [ ]:
# define theta series, bioen performs for each theta reweighting
theta_series = [10e5, 10e4, 10e3, 10e2, 10e1, 10e0, 10e-1, 0]
np.savetxt('theta.dat', np.array(theta_series))

## Run BioEn with bash script and use files generated in the preparation

In [ ]:
def run_bioen_script(label_pair_id, nmodels):
    ! bash run_bioen_ipynb_deer.sh $label_pair_id $nmodels 
    return

In [ ]:
%%capture c
for spin_label_pair in spin_label_pairs:
    label_pair_id = '{}-{}'.format(spin_label_pair[0][0], spin_label_pair[1][0])
    # start BioEn reweighting for each spin-label pair
    run_bioen_script(label_pair_id, nmodels)

# Analysis

## Settings and input parameters for analysis of BioEn outputs

In [ ]:
def load_bioen_pkl():
    """
    Loads output of BioEn pkl file (contains all information)
    """
    df = pd.read_pickle("{}/bioen_results.pkl".format(path_output_bioen))
    return df.to_dict()


In [ ]:
bioen_data = load_bioen_pkl()

`bioen_data` contains all needed and useful information to analyse the BioEn calculations. <br>
For each previously defined confidence value $\theta$ (defined in the variable `theta_series`) a list of BioEn output is available:

**Experimental information**
- **`exp`** <br> 
--> experimental DEER signals for each label <br>
--> in case of DEER data, this variable contains time, DEER signal and polynomial fit of the DEER signal <br>
--> time: `data['exp']['370-292'][:,0]` <br>
--> DEER trace: `data['exp']['370-292'][:,1]` <br>
--> polynomial fit of DEER trace: `data['exp']['370-292'][:,1]` <br>
- **`exp_err`** <br>
--> error/noise of each experimental data point defined in `exp`
- **`labels`** <br> 
--> list of labels used in the refinement
- **`nrestraints`** <br>
--> number of data points ($M$)

**Simulated data and parameters**
- **`sim_init`** <br> 
--> initial ensemble averaged simulated DEER traces for each label (see above for exp)
- **`sim_wopt`** <br>
--> optimized ensemble averaged simulated DEER traces for each label and for a given confidence value $\theta$
- **`moddepth`** <br>
--> modulation depth of the optimized ensemble averaged simulated DEER traces for each label 
- **`nmodels_list`** <br>
--> list of model ids (extracted e.g. from models_370-292.dat)
- **`nmodels`** <br>
--> number of models ($N$), which is also the number of ensemble members (relevant in the BioEn context) <br>
--> `nmodels` = `nrotamers` <br>
--> number of rotamers, which is needed to backcalculate weights of rotamers of a spin-label position (relevant in the analysis contex) <br>

**Free energy parameters**
- **`theta`** <br>
--> confidence parameter $\theta$
- **`chi2_init`** <br>
--> deviation from the experimental data with the initial weight distribution <br>
--> if you use `chi2_init` of the maximal value of $\theta$, $\chi^2$ is calculated with the prior weight distribution `w0`
- **`chi2`** <br>
--> optimal $\chi^2$ calculated with optimal weight distribution `wopt`
- **`S_init`** <br>
--> initial relative entropy
--> if you use `S_init` of the maximal value of $\theta$, $S$ is equal to $0$
- **`S`** <br>
--> relative entropy of optimized weight distribution
- **`w0`** <br>
--> reference weight distribution
- **`winit`** <br>
--> initial weight distribution
- **`wopt`** <br>
--> optimal weight distribution <br>
--> optimized for each $\theta$

**Misc**
- **`len_sim`** <br>
--> seconds it took for BioEn to optimize for a given value of $\theta$
- **`optimization_method`** <br>
--> information about the optimization method: log-weights or forces
- **`optimization_minimizer`** <br>
--> information about the minimer: scipy, GSL, LBFGS
- **`optimization_algorithm`** <br>
--> information about the inimizer: <br>
--> scipy: bfgs, lbfgs, cg <br>
--> GSL: conjugate_fr, conjugate_pr, bfgs, bfgs, steepest_descent<br>
--> LBFGS: lbfgs <br>

### Plot $\chi^{2} vs. S_{\mathrm{KL}}$

the lower $\theta$, the closer to experimental data

In [ ]:
def visualize_chi2_skl(bioen_data, label_pair_id):
    fs = 22
    fig = plt.figure(figsize=[6,4])
    ax = fig.add_subplot(111)
    
    for theta in theta_series:
        chi2 = bioen_data[theta]['chi2'] / bioen_data[theta]['nrestraints']
        skl = - bioen_data[theta]['S']
        ax.scatter(skl, chi2, marker='^', s=160, label=theta)  
    
    #ax.set_xticks(range(-1,6,1))   
    #ax.set_xticklabels(range(-1,6,1), fontsize=fs)
    #ax.set_xlim(-0.4,5.5)
    ax.set_xlabel(r'$S_{\mathrm{KL}}$', fontsize=fs)

    #ax.set_yticks(np.arange(0,10,1))   
    #ax.set_yticklabels(np.arange(0,10,1), fontsize=fs)
    #ax.set_ylim(-0.2, 4)
    ax.set_ylabel(r'$\chi^{2}$', fontsize=fs)    
    
    plt.grid()
    
    ax.legend(ncol=2, fontsize=10)
    
    plt.tight_layout()
    plt.savefig("{}/bioen_chi2_skl.png".format(path_output_analysis, label_pair_id), dpi=400)
    plt.show()
    return

In [ ]:
for spin_label_pair in spin_label_pairs:
    label_pair_id = '{}-{}'.format(spin_label_pair[0][0], spin_label_pair[1][0])
    visualize_chi2_skl(bioen_data, label_pair_id)

## Comparison experimental and weighted ensemble averaged DEER traces

In [ ]:
def visualize_deer_traces(theta, bioen_data, label_pair_id):
    fs = 22
    fig = plt.figure(figsize=[6,4])
    ax = fig.add_subplot(111)
    
    theta_max = np.max(list(bioen_data.keys()))
    
    label = list(bioen_data[theta]['exp']['deer'].keys())[0]

    exp = bioen_data[theta]['exp']['deer'][label]
    exp_err = bioen_data[theta]['exp_err']['deer'][label]
    
    sim_init = bioen_data[theta_max]['sim_init']['deer'][label]
    sim = bioen_data[theta]['sim_wopt']['deer'][label]
    
    ax.plot(exp[:,0], exp[:,1], color='black', linewidth=2.5, label='Exp.',zorder=2)
    ax.fill_between(exp[:,0], exp[:,2]-exp_err, exp[:,2]+exp_err, color='black', alpha=0.2, label='Error', linewidth=1.0, zorder=1)
    ax.plot(exp[:,0], sim_init, color='green', linewidth=3.0, label=r"X-ray",zorder=0)
    ax.plot(exp[:,0], sim, color='red', linewidth=3.0, label=r"BioEn".format(theta),zorder=3)
    
    ax.set_xticks(np.arange(0,4.0,1.0))
    ax.set_xticklabels(np.arange(0,4.0,1.0), fontsize=fs)
    ax.set_xlim(0,3.5)
    ax.set_xlabel(r't [$\mu$s]', fontsize=fs+2)

    ax.set_yticks(np.arange(-0.6,1.4,0.2))
    ax.set_yticklabels(np.arange(-0.6,1.4,0.2), fontsize=fs)
    ax.set_ylim(0.35,1.0)
    ax.set_ylabel(r'F(t)', fontsize=fs)

    ax.legend(loc=1, ncol=2, fontsize=10)
    
    plt.grid()
    
    plt.tight_layout()
    plt.savefig("{}/bioen_deer_{}.png".format(path_output_analysis,label_pair_id), dpi=400)
    plt.show()
    return

In [ ]:
theta = 100.0
for spin_label_pair in spin_label_pairs:
    label_pair_id = '{}-{}'.format(spin_label_pair[0][0], spin_label_pair[1][0])
    visualize_deer_traces(theta, bioen_data, label_pair_id)

## Cummulative weight

In [ ]:
def visualize_cum_dist(theta, bioen_data, label_pair_id):
    fs = 22
    fig = plt.figure(figsize=[6,4])
    ax = fig.add_subplot(111)
    
    nmodels = bioen_data[theta]["nmodels"]
    a = np.vstack((bioen_data[theta]["nmodels_list"], np.array(bioen_data[theta]["wopt"]).reshape(1,-1)))
    
    models = []
    for i, id in enumerate(np.argsort(a[1,:])[::-1]):
        if i == 0: models.append([a[:,id][0], float(a[:,id][1]), float(a[:,id][1])])
        else: models.append([a[:,id][0], float(a[:,id][1]), models[-1][2]+float(a[:,id][1])])

    models = np.array(models)
    ax.scatter(range(1,nmodels+1), models[:,2], s=20, color='green', zorder=2)
    ax.plot(range(1,nmodels+1), models[:,2], color='black', zorder=1)

    ax.set_xticks([0.0, 10.0, 100.0, 1000.0, 10000.0])
    ax.set_xticklabels([r'$10^{0}$', r'$10^{1}$', r'$10^{2}$', r'$10^{3}$', r'$10^{4}$'], fontsize=fs)
    ax.set_xlabel("Fraction of configurations", fontsize=fs)
    ax.semilogx()
    ax.set_yticklabels(np.arange(0,1.2,0.2), fontsize=fs)
    ax.set_ylim(0,1)
    ax.set_ylabel("Cumulative distribution", fontsize=fs)
    
    ax.grid()
    
    plt.savefig('{}/bioen_cum_weights_{}.png'.format(path_output_analysis, label_pair_id), dpi=600, bbox_inches='tight')
    return

In [ ]:
theta = 100.0
for spin_label_pair in spin_label_pairs:
    label_pair_id = '{}-{}'.format(spin_label_pair[0][0], spin_label_pair[1][0])
    visualize_cum_dist(theta, bioen_data, label_pair_id)

## Distance distribution

### Obtain distances based on the weights from the BioEn calculation and frames*pkl

In [ ]:
def get_distances():
    distances_fn = "{}/distances.pkl".format(path_output_preparation)
    with open(distances_fn, 'rb') as fp:
        distances = pickle.load(fp)
    return distances

In [ ]:
def combine_distances_weights(theta, distances, label_pair_id):
    nmodels = bioen_data[theta]["nmodels"]
    weights = np.array(bioen_data[theta]["wopt"])
    nmodels_weights = (np.vstack((bioen_data[theta]["nmodels_list"], 
                       np.array(bioen_data[theta]["wopt"]).reshape(1,-1)))).T
    for model in nmodels_weights:
        a = np.array([distances[model[0]][label_pair_id], [model[1]]* len(distances[model[0]][label_pair_id])])
        if model[0] == 0: weights_distances = a
        else: weights_distances = np.column_stack((weights_distances, a))

    return np.array(weights_distances)

In [ ]:
distances = get_distances()
theta = 100.0
weights_distances_all = dict()
for spin_label_pair in spin_label_pairs:
    label_pair_id = '{}-{}'.format(spin_label_pair[0][0], spin_label_pair[1][0])
    weights_distances_all[label_pair_id] = combine_distances_weights(theta, distances, label_pair_id)

### Visulize distances with prior and optimized weights

In [ ]:
def visualize_dist_distr(weights_distances):
    fs = 22
    fig = plt.figure(figsize=[6,4])
    ax = fig.add_subplot(111)
    
    distances = weights_distances[0,:]
    weights = weights_distances[1,:]
    
    ax.axes.hist(distances, bins=100, color='green', label="X-ray (w0)", alpha=0.2, zorder=2, normed=True)
    ax.axes.hist(distances, bins=100, color='green', alpha=0.8, zorder=2, normed=True, histtype='step')
    ax.axes.hist(distances, weights=weights, bins=100, color='red', label=r"Shift (wopt, $\theta=10^3$)", alpha=0.2, normed=True,)
    ax.axes.hist(distances, weights=weights, bins=100, color='red', alpha=0.8, normed=True, histtype='step')
    
    ax.set_xticks(range(0,80,10))
    ax.set_xticklabels(range(0,80,10), fontsize=fs)
    ax.set_xlim(15, 60)
    ax.set_xlabel(r'r [nm]', fontsize=fs)

    ax.set_yticklabels([], fontsize=fs)
    ax.set_ylabel(r'P(r)', fontsize=fs)    
    
    plt.grid(zorder=3)
    
    plt.savefig('{}/bioen_dist_distributions.png'.format(path_output_analysis), dpi=600, bbox_inches='tight')
    
    return

In [ ]:
for spin_label_pair in spin_label_pairs:
    label_pair_id = '{}-{}'.format(spin_label_pair[0][0], spin_label_pair[1][0])
    visualize_dist_distr(weights_distances_all[label_pair_id])